In [74]:
import cv2
import numpy as np
import os

In [75]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [76]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [77]:
def nonmax_suppression(image):

    NONMAX_SUPPRESSION_KERNEL = 16
    STRIDE = 8
    
    result = np.zeros((image.shape))
    
    _image = np.lib.stride_tricks.sliding_window_view(image, NONMAX_SUPPRESSION_KERNEL, 1)[:,::STRIDE]
    _image = np.lib.stride_tricks.sliding_window_view(_image, NONMAX_SUPPRESSION_KERNEL, 0)[::STRIDE,:]

    _image = np.reshape(_image,(*_image.shape[:2],-1))
    _image = np.argmax(_image, axis=2)

    mg = np.mgrid[0:_image.shape[0],0:_image.shape[1]]

    # y offset: mg[0], x offset: mg[1]
    index_y = _image // NONMAX_SUPPRESSION_KERNEL + mg[0] * STRIDE 
    index_x = _image % NONMAX_SUPPRESSION_KERNEL + mg[1] * STRIDE 
    index_x, index_y = index_x.flatten(), index_y.flatten()


    for i, j in zip(index_y, index_x):
        result[i][j]=image[i][j]  
     
    return  result

In [94]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 5
    THRESHOLD = 500
    K = 0.04

    #  Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (3,3),3)
    # gradient
    I_x = np.gradient(Gau_img, axis=1) 
    I_y = np.gradient(Gau_img, axis=0) 


    # Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    
    # Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255

    # Nonmax Suppression
    R = nonmax_suppression(R)
    
    # 濾邊界
    R[:20 , :] = 0
    R[: ,:20 ] = 0
    R[-20:, :] = 0
    R[:,-20: ] = 0
    # return mask
    return R
        

In [95]:
a = load_images_from_folder('img/test')
R = detect_Harris_corner(a[0])

In [96]:
show_image_by_OpenCV(R)

In [81]:
def feature_descriptor(feature_mask):
    # descriptor size: 40*40
    feature_point_x, feature_point_y = np.nonzero(feature_mask)
    # 5*5 sum/mean -> 8*8(descriptor matrix)
    
    feature_mask[feature_point_x][feature_point_y]
    
    pass